<a href="https://colab.research.google.com/github/PCBZ/CS6140/blob/main/HW5/HW5_problem2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
%pip install ucimlrepo

from ucimlrepo import fetch_ucirepo
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

import numpy as np

class LinearSVM:
    """
    SVM implementation using Sequential Minimal Optimization (SMO) algorithm
    """
    def __init__(self, C=1.0, tol=1e-3, max_pass=200):
        """
        Initialize SVM with Linear Kernel

        Parameters
        ----------
        C : float, optional (default=1.0)
            Penalty parameter C of the error term.
        tol : float, optional (default=1e-3)
            Tolerance for stopping criteria.
        max_pass : int, optional (default=5)
            Maximum number of passes through the training data.
        """
        self.C = C
        self.tol = tol
        self.max_pass = max_pass

    def kernel_function(self, x1, x2):
        """
        Use liner kernel function
        """
        return x1 @ x2

    def compute_kernel_matrix(self, X1, X2=None):
        """
        Compute the kernel matrix for a given set of vectors.
        """
        if X2 is None:
            X2 = X1
        m = X1.shape[0]
        n = X2.shape[0]
        K = np.zeros((m, n))
        for i in range(m):
            for j in range(n):
                K[i, j] = self.kernel_function(X1[i], X2[j])
        return K

    def predict_single(self, x):
        """
        Predict the label of a single vector.
        """
        prediction = self.bias
        for i in range(self.num_samples):
            if self.alphas[i] > 0:
                prediction += self.alphas[i] * self.y[i] * self.kernel_function(self.X[i], x)
        return prediction

    def get_error(self, i):
        """
        Compute the error of a single vector.
        """
        return self.predict_single(self.X[i]) - self.y[i]

    def select_second_alpha(self, first_idx, first_error):
        """
        Select the second alpha to optimize.
        """
        return (first_idx + 1) % self.num_samples


    def optimize_alpha_pair(self, first_idx, second_idx):
        """
        Optimize alpha pair using SMO algorithm
        """
        if first_idx == second_idx:
            return 0
        alpha1 = self.alphas[first_idx]
        alpha2 = self.alphas[second_idx]
        y1 = self.y[first_idx]
        y2 = self.y[second_idx]

        error1 = self.get_error(first_idx)
        error2 = self.get_error(second_idx)

        # Calculate left and right boundary
        if y1 != y2:
            L = max(0, alpha2 - alpha1)
            H = min(self.C, self.C + alpha2 - alpha1)
        else:
            L = max(0, alpha1 + alpha2 - self.C)
            H = min(self.C, alpha1 + alpha2)
        if L == H:
            return 0

        kernel11 = self.kernel_function(self.X[first_idx], self.X[first_idx])
        kernel22 = self.kernel_function(self.X[second_idx], self.X[second_idx])
        kernel12 = self.kernel_function(self.X[first_idx], self.X[second_idx])

        # Calculate second derivative
        eta = kernel11 + kernel22 - 2 * kernel12

        if eta > 0:
            # Update alpha2
            alpha2_new = alpha2 + y2 * (error1 - error2) / eta
            alpha2_new = np.clip(alpha2_new, L, H)
        else:
            # Cannot optimize
            return 0

        if np.abs(alpha2_new - alpha2) < self.tol:
            # Too small change on alpha2
            return 0

        alpha1_new = alpha1 + y1 * y2 * (alpha2 - alpha2_new)

        self.alphas[first_idx] = alpha1_new
        self.alphas[second_idx] = alpha2_new

        bias1 = self.bias - error1 - y1 * (alpha1_new - alpha1) * kernel11 - y2 * (alpha2_new - alpha2) * kernel12
        bias2 = self.bias - error2 - y1 * (alpha1_new - alpha1) * kernel12 - y2 * (alpha2_new - alpha2) * kernel22

        if 0 < alpha1_new < self.C:
            self.bias = bias1
        elif 0 < alpha2_new < self.C:
            self.bias = bias2
        else:
            self.bias = (bias1 + bias2) / 2

        return 1

    def examine_sample(self, sample_idx):
        """
        Examine sample for KKT violations and trigger optimization if needed.
        """
        y = self.y[sample_idx]
        alpha = self.alphas[sample_idx]
        error = self.get_error(sample_idx)

        # Check KTT violations
        if ((y * error < -self.tol and alpha < self.C) or (y * error > self.tol and alpha > 0)):
            # Select second alpha
            second_idx = self.select_second_alpha(sample_idx, error)
            if self.optimize_alpha_pair(sample_idx, second_idx):
                # If success, return 1
                return 1
        # Not optimized
        return 0

    def fit(self, X, y):
        """
        Train the SVM model to the training data.
        """
        self.X = X
        self.y = y
        self.num_samples, self.num_features = X.shape
        # Lagrange multipler
        self.alphas = np.zeros(self.num_samples)
        self.bias = 0

        num_passes = 0
        while num_passes < self.max_pass:
            num_changed = 0

            for i in range(self.num_samples):
                num_changed += self.examine_sample(i)

            if num_changed == 0:
                # No alpha changed, convergence, quit loop
                break
            else:
                num_passes += 1

            print(f"Pass {num_passes}: {num_changed} alphas changed")

    def predict(self, X):
        """
        Predict the labels of the input data.
        """
        predictions = np.zeros(X.shape[0])
        for i in range(X.shape[0]):
            predictions[i] = self.predict_single(X[i])
        return np.sign(predictions)


def fetch_spambase_data():
    """
    Fetch Spambase dataset from UCI repository
    """
    spambase = fetch_ucirepo(id=94)

    # Extract features and targets
    X = spambase.data.features.values
    y = spambase.data.targets.values.ravel()

    return X, y

def preprocess_data(X, y):
    """
    Preprocess the data
    """
    # Covert labels to 1/-1
    y = np.where(y == 0, -1, 1)

    # Split data
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Standardization
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    return X_train, X_test, y_train, y_test

def train_and_evaluate_svm_model(X_train, y_train, X_test, y_test):
    """
    Train and evaluate SVM model
    """
    model = LinearSVM(C=2.0, tol=0.005, max_pass=200)
    model.fit(X_train, y_train)

    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)

    train_accuracy = accuracy_score(y_train, y_train_pred)
    test_accuracy = accuracy_score(y_test, y_test_pred)

    print(f"Train Accuracy: {train_accuracy}")
    print(f"Test Accuracy: {test_accuracy}")


if __name__ == "__main__":
    X, y = fetch_spambase_data()
    X_train, X_test, y_train, y_test = preprocess_data(X, y)
    train_and_evaluate_svm_model(X_train, y_train, X_test, y_test)




Pass 1: 200 alphas changed
Pass 2: 211 alphas changed
Pass 3: 180 alphas changed
Pass 4: 185 alphas changed
Pass 5: 176 alphas changed
Pass 6: 181 alphas changed
Pass 7: 172 alphas changed
Pass 8: 175 alphas changed
Pass 9: 154 alphas changed
Pass 10: 154 alphas changed
Pass 11: 148 alphas changed
Pass 12: 150 alphas changed
Pass 13: 151 alphas changed
Pass 14: 138 alphas changed
Pass 15: 138 alphas changed
Pass 16: 138 alphas changed
Pass 17: 149 alphas changed
Pass 18: 143 alphas changed
Pass 19: 142 alphas changed
Pass 20: 138 alphas changed
Pass 21: 150 alphas changed
Pass 22: 145 alphas changed
Pass 23: 141 alphas changed
Pass 24: 136 alphas changed
Pass 25: 134 alphas changed
Pass 26: 133 alphas changed
Pass 27: 130 alphas changed
Pass 28: 123 alphas changed
Pass 29: 117 alphas changed
Pass 30: 111 alphas changed
Pass 31: 115 alphas changed
Pass 32: 112 alphas changed
Pass 33: 113 alphas changed
Pass 34: 119 alphas changed
Pass 35: 114 alphas changed
Pass 36: 115 alphas changed
P